In [1]:
%matplotlib inline
import pandas as pd
import os
import numpy as np

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [13]:
pdX = pd.read_pickle(os.path.join('..', 'data', 'our_df2.pickled'))
y = pd.read_pickle(os.path.join('..', 'data', 'ytrain.pkl'))

# y has one more predicion - trim it
y = y[0:-1]


In [14]:
print(pdX.shape)
print(len(y)) # y is 1-dim

(17253, 337)
17253


In [15]:
import xgboost as xg

# suggested params
param = {
    'bst:eta' : [0.002, 0.02],
    'bst:subsample' : [0.5,1],
    'bst:mim_child_weight' : [0.1,10],
    'bst:max_depth' : [2,30],
    'bst:gamma' : [0,1],
    'objective':'binary:logistic'
}

# create booster

tb = xg.XGBClassifier(learning_rate=0.002)

In [49]:
# preprocess data here
# our approach has left some Nones, let's do a second pass ;)

def clean(x):
    if x is None:
        return hash(x)
    else:
        return x

pdX = pdX.applymap(clean)
print("done")

done


In [50]:
from sklearn.cross_validation import train_test_split

X_fit, X_eval, y_fit, y_eval= train_test_split(
    pdX, y, test_size=0.2, random_state=1
)

# X = xg.DMatrix(np.array(pdX))
tb.fit(X_fit, y_fit, eval_set=[(X_eval, y_eval)], eval_metric='logloss', verbose=True)
print tb.evals_result()
# tb.fit(X, y)


[0]	validation_0-logloss:0.691426
[1]	validation_0-logloss:0.689711
[2]	validation_0-logloss:0.688004
[3]	validation_0-logloss:0.686303
[4]	validation_0-logloss:0.684609
[5]	validation_0-logloss:0.682922
[6]	validation_0-logloss:0.681241
[7]	validation_0-logloss:0.679567
[8]	validation_0-logloss:0.6779
[9]	validation_0-logloss:0.676239
[10]	validation_0-logloss:0.674585
[11]	validation_0-logloss:0.672937
[12]	validation_0-logloss:0.671296
[13]	validation_0-logloss:0.669661
[14]	validation_0-logloss:0.668032
[15]	validation_0-logloss:0.66641
[16]	validation_0-logloss:0.664795
[17]	validation_0-logloss:0.663185
[18]	validation_0-logloss:0.661582
[19]	validation_0-logloss:0.659985
[20]	validation_0-logloss:0.658395
[21]	validation_0-logloss:0.65681
[22]	validation_0-logloss:0.655231
[23]	validation_0-logloss:0.653659
[24]	validation_0-logloss:0.652093
[25]	validation_0-logloss:0.650532
[26]	validation_0-logloss:0.648978
[27]	validation_0-logloss:0.64743
[28]	validation_0-logloss:0.645887
